<a href="https://colab.research.google.com/github/tada20001/NLP_2023/blob/main/CH9_12_%EB%AC%B8%EC%84%9C_%EC%9E%84%EB%B2%A0%EB%94%A9_%EC%9B%8C%EB%93%9C_%EC%9E%84%EB%B2%A0%EB%94%A9%EC%9D%98_%ED%8F%89%EA%B7%A0(Average_Word_Embedding).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

문장 단어들의 임베딩 벡터들의 평균만으로 텍스트 분류하는 예시

### 1. 데이터 로드와 전처리
-------------------

IMDB 리뷰데이터 사용. 등장빈도 순서 20,000개까지만 사용함

In [1]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
vocab_size = 20000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)
print(X_train.shape)
print(X_test.shape)

17464789/17464789 [==============================] - 0s 0us/step
(25000,)
(25000,)


In [4]:
print(X_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [5]:
print(y_train[0])

1


In [8]:
## 평균길이와 최대 길이 확인
print('훈련용 리뷰의 평균길이 :', np.mean([len(word) for word in X_train]))
print('테스트용 리뷰의 평균길이 :', np.mean([len(word) for word in X_test]))
print('훈련용 리뷰의 최대길이 :', max(len(word) for word in X_train))
print('테스트용 리뷰의 최대길이 :', max(len(word) for word in X_test))


훈련용 리뷰의 평균길이 : 238.71364
테스트용 리뷰의 평균길이 : 230.8042
훈련용 리뷰의 최대길이 : 2494
테스트용 리뷰의 최대길이 : 2315


In [9]:
# 여기서는 최대 리뷰길이를 400으로 설정하여 패딩 적용
max_len = 400

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_train.shape, X_test.shape

((25000, 400), (25000, 400))

### 2. 모델 설계
--------------------
GlobalAveragePoolingID()은 입력으로 들어오는 모든 벡터들의 평균을 구하는 역할을 함. Embedding() 다음에 GlobalAveragePooling1D()를 추가하면 해당 문장의 모든 단어 벡터들의 평균벡터를 구함

이진 분류를 수행해야 하므로 그 후에는 시그모이드 함수를 활성화함수로 사용하는 뉴런 1개를 배치함

* embedding -> average -> activation function -> compile -> training(fit)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [11]:
embedding_dim = 64

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))
model.add(GlobalAveragePooling1D())
model.add(Dense(units=1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('embedding_average_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, batch_size=32, epochs=10, callbacks=[es, mc], validation_split=0.2)

Epoch 1/10
625/625 [==============================] - ETA: 0s - loss: 0.6288 - acc: 0.7171
Epoch 1: val_acc improved from -inf to 0.81660, saving model to embedding_average_model.h5
625/625 [==============================] - 16s 25ms/step - loss: 0.6288 - acc: 0.7171 - val_loss: 0.5217 - val_acc: 0.8166
Epoch 2/10
624/625 [============================>.] - ETA: 0s - loss: 0.4302 - acc: 0.8563
Epoch 2: val_acc improved from 0.81660 to 0.87040, saving model to embedding_average_model.h5
625/625 [==============================] - 16s 26ms/step - loss: 0.4301 - acc: 0.8563 - val_loss: 0.3785 - val_acc: 0.8704
Epoch 3/10
623/625 [============================>.] - ETA: 0s - loss: 0.3188 - acc: 0.8910
Epoch 3: val_acc improved from 0.87040 to 0.87960, saving model to embedding_average_model.h5
625/625 [==============================] - 14s 22ms/step - loss: 0.3185 - acc: 0.8910 - val_loss: 0.3234 - val_acc: 0.8796
Epoch 4/10
625/625 [==============================] - ETA: 0s - loss: 0.2616 - 

학습이 종료된 후에 테스트 데이터로 검증 수행

In [13]:
from tensorflow.keras.models import load_model
loaded_model = load_model('embedding_average_model.h5')

print('\n 테스트 정확도: %.4f' % (loaded_model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 2s 2ms/step - loss: 0.2917 - acc: 0.8866

 테스트 정확도: 0.8866


신경망을 별도로 추가하지 않아도 88% 이상의 정확도를 얻어냄